In [36]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import AIMessage,SystemMessage,HumanMessage
from langchain.chains.llm import LLMChain


# Initialize the chatbot
chat = ChatOpenAI(
    api_key = os.environ['ZHIPUAI_API_KEY'],
    base_url = os.environ['ZHIPUAI_BASE_URL'],
    model = os.environ['ZHIPUAI_MODEL_4'],
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()],
)



In [37]:
message = [
    SystemMessage(content="You are a helpful assistant!"),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Java.")
]

In [38]:
res = chat(message)
res

Java who?

AIMessage(content='Java who?', response_metadata={'finish_reason': 'stop', 'model_name': 'glm-4-0520'}, id='run-d4726934-c088-4b45-bb67-943ae7172602-0')

### 处理 LLM 存在的缺陷

    1. 容易出现幻觉
    2. 信息滞后
    3. 专业领域深度知识缺乏

In [39]:
message = [
    SystemMessage(content="你是一个专业知识助手"),
    HumanMessage(content="你是否知道 JimiAI 模型?")
]

res = chat(message)
res

JimiAI 模型并不是一个广为人知的模型，至少在我所掌握的知识范围内，它不是一个著名的或者广泛使用的AI模型。可能是因为这个名字是某个特定公司、项目或者研究小组内部使用的名称，而没有在更广泛的AI领域内获得显著的知名度。如果JimiAI模型是某个特定领域或项目中的概念，可能需要更多的上下文信息才能提供准确的信息。如果您能提供更多关于JimiAI模型的背景信息，我会尽力帮助您了解相关内容。

AIMessage(content='JimiAI 模型并不是一个广为人知的模型，至少在我所掌握的知识范围内，它不是一个著名的或者广泛使用的AI模型。可能是因为这个名字是某个特定公司、项目或者研究小组内部使用的名称，而没有在更广泛的AI领域内获得显著的知名度。如果JimiAI模型是某个特定领域或项目中的概念，可能需要更多的上下文信息才能提供准确的信息。如果您能提供更多关于JimiAI模型的背景信息，我会尽力帮助您了解相关内容。', response_metadata={'finish_reason': 'stop', 'model_name': 'glm-4-0520'}, id='run-88066a3c-86e3-460d-9632-b39e3e3ec9be-0')

In [40]:
JimiAI_Information = [
    "JimiAI 还在研发中,目前还没有什么功能,啥也不是",
    "JimiAI 是个一个法律大模型",
    "JimiAI 不会做任何事情"
]

source_knowledge = '\n'.join(JimiAI_Information)
print(source_knowledge)

JimiAI 还在研发中,目前还没有什么功能,啥也不是
JimiAI 是个一个法律大模型
JimiAI 不会做任何事情


In [41]:
query = "你知道什么是 JimiAI 吗？"
prompt_template = f"""基于一下内容回答问题:
内容:{source_knowledge}
Query:{query}"""

In [42]:
prompt = HumanMessage(content=prompt_template)
message.append(prompt)
res = chat(message)
# print(res.content)
res

JimiAI 是一个正在研发中的法律领域大模型。目前，它还没有实现具体的功能，也就是说，它还不能执行任何任务或操作。从您提供的内容来看，JimiAI 还处于开发的早期阶段。

AIMessage(content='JimiAI 是一个正在研发中的法律领域大模型。目前，它还没有实现具体的功能，也就是说，它还不能执行任何任务或操作。从您提供的内容来看，JimiAI 还处于开发的早期阶段。', response_metadata={'finish_reason': 'stop', 'model_name': 'glm-4-0520'}, id='run-769b4ccd-ddf0-458e-97d8-f422cf64f314-0')

### 创建一个RAG对话
1. 加载数据

    以这篇论文为例：https://arxiv.org/abs/2408.01122<br/>
    https://arxiv.org/pdf/2408.01122.pdf

In [43]:
# pip install pypdf

In [44]:
from langchain.document_loaders import PyPDFLoader,TextLoader

# loader = PyPDFLoader("./2408.01122v1.pdf")
# pages = loader.load_and_split()

loader= TextLoader("./data/doupoch1toch7.txt",encoding='utf8')
pages= loader.load_and_split()

In [45]:
pages[0]

Document(metadata={'source': './data/doupoch1toch7.txt'}, page_content='第一章 陨落的天才\n\n    “斗之力，三段！”\n\n    望着测验魔石碑上面闪亮得甚至有些刺眼的五个大字，少年面无表情，唇角有着一抹自嘲，紧握的手掌，因为大力，而导致略微尖锐的指甲深深的刺进了掌心之中，带来一阵阵钻心的疼痛…\n\n    “萧炎，斗之力，三段！级别：低级！”测验魔石碑之旁，一位中年男子，看了一眼碑上所显示出来的信息，语气漠然的将之公布了出来…\n\n    中年男子话刚刚脱口，便是不出意外的在人头汹涌的广场上带起了一阵嘲讽的骚动。\n\n    “三段？嘿嘿，果然不出我所料，这个“天才”这一年又是在原地踏步！”\n\n    “哎，这废物真是把家族的脸都给丢光了。”\n\n    “要不是族长是他的父亲，这种废物，早就被驱赶出家族，任其自生自灭了，哪还有机会待在家族中白吃白喝。”\n\n    “唉，昔年那名闻乌坦城的天才少年，如今怎么落魄成这般模样了啊？”\n\n    “谁知道呢，或许做了什么亏心事，惹得神灵降怒了吧…”\n\n    周围传来的不屑嘲笑以及惋惜轻叹，落在那如木桩待在原地的少年耳中，恍如一根根利刺狠狠的扎在心脏一般，让得少年呼吸微微急促。\n\n    少年缓缓抬起头来，露出一张有些清秀的稚嫩脸庞，漆黑的眸子木然的在周围那些嘲讽的同龄人身上扫过，少年嘴角的自嘲，似乎变得更加苦涩了。\n\n    “这些人，都如此刻薄势力吗？或许是因为三年前他们曾经在自己面前露出过最谦卑的笑容，所以，如今想要讨还回去吧…”苦涩的一笑，萧炎落寞的转身，安静的回到了队伍的最后一排，孤单的身影，与周围的世界，有些格格不入。\n\n    “下一个，萧媚！”\n\n    听着测验人的喊声，一名少女快速的人群中跑出，少女刚刚出场，附近的议论声便是小了许多，一双双略微火热的目光，牢牢的锁定着少女的脸颊…\n\n    少女年龄不过十四左右，虽然并算不上绝色，不过那张稚气未脱的小脸，却是蕴含着淡淡的妩媚，清纯与妩媚，矛盾的集合，让得她成功的成为了全场瞩目的焦点…\n\n    少女快步上前，小手轻车熟路的触摸着漆黑的魔石碑，然后缓缓闭上眼睛…\n\n    在少女闭眼片刻之后，漆黑的魔石碑之上再次亮起了光芒

2. 知识切片，将文档分割成均匀的块，每个块是一段原始文本。

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
'''
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
# text_splitter = CharacterTextSplitter(
#     separator = '\n\n\n',
#     chunk_size=450,
#     chunk_overlap=50,
#     length_function=len,
#     is_separator_regex=False,
# )
docs = text_splitter.split_documents(pages)
len(docs)

59

In [47]:
from __future__ import annotations

import logging
from typing import Dict, List, Any

from langchain.embeddings.base import Embeddings
from langchain.pydantic_v1 import BaseModel, root_validator

logger = logging.getLogger(__name__)

class ZhipuAIEmbeddings(BaseModel, Embeddings):
    """`Zhipuai Embeddings` embedding models."""

    client: Any
    """`zhipuai.ZhipuAI"""

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """
        实例化ZhipuAI为values["client"]

        Args:

            values (Dict): 包含配置信息的字典，必须包含 client 的字段.
        Returns:

            values (Dict): 包含配置信息的字典。如果环境中有zhipuai库，则将返回实例化的ZhipuAI类；否则将报错 'ModuleNotFoundError: No module named 'zhipuai''.
        """
        from zhipuai import ZhipuAI
        values["client"] = ZhipuAI()
        return values
    
    def _embed(self, texts: str) -> List[float]:
        embeddings = self.client.embeddings.create(
            model="embedding-3",
            input=texts
        )
        return embeddings.data[0].embedding
    

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.
        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """
        return [self._embed(text) for text in texts]
    
    
    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        resp = self.embed_documents([text])
        return resp[0]



    async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
        """Asynchronous Embed search docs."""
        raise NotImplementedError("Please use `embed_documents`. Official does not support asynchronous requests")

    async def aembed_query(self, text: str) -> List[float]:
        """Asynchronous Embed query text."""
        raise NotImplementedError("Please use `aembed_query`. Official does not support asynchronous requests")

3. 利用 embedding 模型对每个文本片段进行向量化，并存到向量数据库中。

In [48]:
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings import QianfanEmbeddingsEndpoint
# from langchain.embeddings.xinference import XinferenceEmbeddings
# from langchain.embeddings.base import Embeddings
import os
import chromadb
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
# from zhipuembedding import ZhipuAIEmbeddings

# bge_embeddings = embedding_functions.OpenAIEmbeddingFunction(
#     api_base=os.environ["ZHIPUAI_API_URL"],
#     api_key=os.environ["ZHIPUAI_API_KEY"],
#     model_name= "embedding-3"
# )
embedding_model = ZhipuAIEmbeddings()

# document_ebeddings = []
# for text in docs:
#     document_ebeddings.append(bge_embeddings(text))

#测试，如有需要，删除数据库里的实验用cellection，清除之前的实验内容
dbclient = chromadb.PersistentClient(path='./chorma_test_db')
dbclient.delete_collection(name="zhipu_embedding")

# dbclient = chromadb.PersistentClient(path='./chorma_test_db')
vector_store = Chroma.from_documents(documents=docs, embedding=embedding_model,persist_directory="./chorma_test_db", collection_name="zhipu_embedding")


In [49]:
#测试，如有需要，删除数据库里的实验用cellection，清除之前的实验内容
# dbclient = chromadb.PersistentClient(path='./chorma_test_db')
# dbclient.delete_collection(name="zhipu_embedding")

In [50]:
vector_store.persist()
print(f"向量库中存储的数量：{vector_store._collection.count()}")

向量库中存储的数量：59


In [51]:
query = "萧炎是谁?"
'''
在向量存储中进行相似度搜索。
query 参数是输入的查询字符串，即 "What can CFBench do?"。
k=2 参数指定了返回结果的数量，这里设置为2，表示希望返回最相似的2个结果。
'''
result =  vector_store.similarity_search(query, k=2)

In [52]:
result

[Document(metadata={'source': './data/doupoch1toch7.txt'}, page_content='“他们是云岚宗的人？”萧炎惊讶的低声道。\n\n    虽然并没有外出历练，不过萧炎在一些书籍中却看过有关这剑派的资料，萧家所在的城市名为乌坦城，乌坦城隶属于加玛帝国，虽然此城因为背靠魔兽山脉的地利，而跻身进入帝国的大城市之列，不过也仅仅只是居于末座。\n\n    萧炎的家族，在乌坦城颇有份量，不过却也并不是唯一，城市中，还有另外两大家族实力与萧家相差无几，三方彼此明争暗斗了几十年，也未曾分出胜负…\n\n    如果说萧家是乌坦城的一霸，那么萧炎口中所说的云岚宗，或许便应该说是整个加玛帝国的一霸！这之间的差距，犹如鸿沟，也难怪连平日严肃的父亲，在言语上很是敬畏。\n\n    “他们来我们家族做什么？”萧炎有些疑惑的低声询问道。\n\n    移动的纤细指尖微微一顿，熏儿沉默了一会，方才道：“或许和萧炎哥哥有关…”\n\n    “我？我可没和他们有过什么交集啊？”闻言，萧炎一怔，摇头否认。\n\n    “知道那少女叫什么名字吗？”熏儿淡淡的扫了一眼对面的娇贵少女。'),
 Document(metadata={'source': './data/doupoch1toch7.txt'}, page_content='“萧炎，这里哪有你说话的份？给我闭嘴！”脸色一沉，一位长老怒喝道。\n\n    “萧炎，退下去吧，我知道你心里不好受，不过这里我们自会做主！”另外一位年龄偏大的老者，也是淡淡的道。\n\n    “三位长老，如果今天他们悔婚的对象是你们的儿子或者孙子，你们还会这么说么？”萧炎缓缓站起身子，嘴角噙着嘲讽，笑问道，三位长老对他的不屑是显而易见，所以他也不必在他们面前装怂。\n\n    “你…”闻言，三位长老一滞，脾气暴躁的三长老，更是眼睛一瞪，斗气缓缓附体。\n\n    “三位长老，萧炎哥哥说得并没有错，这事，他是当事人，你们还是不要跟着参合吧。”少女轻灵的嗓音，在厅中淡然的响起。\n\n    听着少女的轻声，三位长老的气焰顿时消了下来，无奈的对视了一眼，旋即点了点头。\n\n    望着萎靡的三位长老，萧炎回转过头，深深的凝视了一眼笑吟吟的萧薰儿，你这妮子，究竟是什么身份？怎么让得三位长老如此忌惮…

5. 原始 query 与检索得到的文本组合起来输入到语言模型，得到最终回答。

In [53]:
def augment_prompt(query: str):
    result = vector_store.similarity_search(query,k=10)
    source_knowledge = "\n".join([x.page_content for x in result])
    augmented_prompt = f"""基于一下内容回答问题:
    内容:{source_knowledge}
    query:{query}
    """
    return augmented_prompt

In [54]:
print(augment_prompt(query))

基于一下内容回答问题:
    内容:“他们是云岚宗的人？”萧炎惊讶的低声道。

    虽然并没有外出历练，不过萧炎在一些书籍中却看过有关这剑派的资料，萧家所在的城市名为乌坦城，乌坦城隶属于加玛帝国，虽然此城因为背靠魔兽山脉的地利，而跻身进入帝国的大城市之列，不过也仅仅只是居于末座。

    萧炎的家族，在乌坦城颇有份量，不过却也并不是唯一，城市中，还有另外两大家族实力与萧家相差无几，三方彼此明争暗斗了几十年，也未曾分出胜负…

    如果说萧家是乌坦城的一霸，那么萧炎口中所说的云岚宗，或许便应该说是整个加玛帝国的一霸！这之间的差距，犹如鸿沟，也难怪连平日严肃的父亲，在言语上很是敬畏。

    “他们来我们家族做什么？”萧炎有些疑惑的低声询问道。

    移动的纤细指尖微微一顿，熏儿沉默了一会，方才道：“或许和萧炎哥哥有关…”

    “我？我可没和他们有过什么交集啊？”闻言，萧炎一怔，摇头否认。

    “知道那少女叫什么名字吗？”熏儿淡淡的扫了一眼对面的娇贵少女。
“萧炎，这里哪有你说话的份？给我闭嘴！”脸色一沉，一位长老怒喝道。

    “萧炎，退下去吧，我知道你心里不好受，不过这里我们自会做主！”另外一位年龄偏大的老者，也是淡淡的道。

    “三位长老，如果今天他们悔婚的对象是你们的儿子或者孙子，你们还会这么说么？”萧炎缓缓站起身子，嘴角噙着嘲讽，笑问道，三位长老对他的不屑是显而易见，所以他也不必在他们面前装怂。

    “你…”闻言，三位长老一滞，脾气暴躁的三长老，更是眼睛一瞪，斗气缓缓附体。

    “三位长老，萧炎哥哥说得并没有错，这事，他是当事人，你们还是不要跟着参合吧。”少女轻灵的嗓音，在厅中淡然的响起。

    听着少女的轻声，三位长老的气焰顿时消了下来，无奈的对视了一眼，旋即点了点头。

    望着萎靡的三位长老，萧炎回转过头，深深的凝视了一眼笑吟吟的萧薰儿，你这妮子，究竟是什么身份？怎么让得三位长老如此忌惮…
“在熏儿四岁到六岁的时候，每天晚上都有人溜进我的房间，然后用一种很是笨拙的手法以及并不雄厚的斗之气，温养我的骨骼与经脉，每次都要弄得自己大汗淋漓后，方才疲惫离开，萧炎哥哥，你说，他会是谁？”熏儿沉默了半晌，忽然的偏过头，对着萧炎嫣然一笑，少女独有的风情，让得周围的少年眼睛有些放光。

    “咳…我，我怎么

In [55]:
# 创建 prompt
prompt = HumanMessage(
    content = augment_prompt(query+"请用中文回答。")
)
message.append(prompt)
res = chat(message)

res

萧炎是小说《斗破苍穹》中的主角，他是一个天赋异禀的青年，出生在乌坦城的萧家，家族在城中颇有份量。萧炎在小说的开始是一个拥有斗气天赋的少年，但后来因为某种原因斗气修为不进反退，被家族视为废物。然而，他凭借坚持不懈的努力和毅力，逐渐成长为一位强大的斗者，并在斗气大陆上展开了一系列的冒险和战斗。

AIMessage(content='萧炎是小说《斗破苍穹》中的主角，他是一个天赋异禀的青年，出生在乌坦城的萧家，家族在城中颇有份量。萧炎在小说的开始是一个拥有斗气天赋的少年，但后来因为某种原因斗气修为不进反退，被家族视为废物。然而，他凭借坚持不懈的努力和毅力，逐渐成长为一位强大的斗者，并在斗气大陆上展开了一系列的冒险和战斗。', response_metadata={'finish_reason': 'stop', 'model_name': 'glm-4-0520'}, id='run-01f6edfb-a2c3-4dfe-b2a8-304f520e0882-0')